Import packages

In [109]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import urllib.request
import camelot

### 116<sup>th</sup> congress

First the twitter handles for the 116<sup>th</sup> congress will be extracted using [this](https://triagecancer.org/congressional-social-media) source. The choice of source comes from the fact that the Twitter handle as well as the party is desired.

`BeautifulSoup` is used to extract the HTML table from the webpage (that has been downloaded to allow for offline work).

In [110]:
# Open data
with open('../Data/Raw/116_congress_twitter.html') as fp:
    soup = BeautifulSoup(fp, 'html.parser')

# Find table
table = soup.find('table', attrs={'id':"footable_16836"})

# Extract data row wise from table
l = []
for tr in table.findAll('tr'):
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

# Make the data into a Pandas data frame and drop irrelevant columns
Data116 = pd.DataFrame(l[1:], columns = [header.getText() for header in table.findAll('th')]).drop(columns = ['Name Links', 'Twitter Links', 'Instagram', 'Facebook Page', 'Facebook'])

# Ensure that the type of politician is alligned
rename_chamber = {'U.S. Representative': 'Representative', 'U.S. Senator': 'Senator'}
Data116 = Data116.replace(rename_chamber).rename(columns = {'Chamber of Congress': 'Type'})

In this data set the state is given as well as congressional district. This is fixed using regex strings as shown below. Moreover the "@" are removed from the Twitter handles as the Twitter API does not need it. The vancant positions in Congress are also disregarded.

In [111]:
# All states abbreviations
us_state_abbrev = {
    r'Alabama.*': 'AL',
    r'Alaska.*': 'AK',
    r'American Samoa.*': 'AS',
    r'Arizona.*': 'AZ',
    r'Arkansas.*': 'AR',
    r'California.*': 'CA',
    r'Colorado.*': 'CO',
    r'Connecticut.*': 'CT',
    r'Delaware.*': 'DE',
    r'District of Columbia.*': 'DC',
    r'Florida.*': 'FL',
    r'Georgia.*': 'GA',
    r'Guam.*': 'GU',
    r'Hawaii.*': 'HI',
    r'Idaho.*': 'ID',
    r'Illinois.*': 'IL',
    r'Indiana.*': 'IN',
    r'Iowa.*': 'IA',
    r'Kansas.*': 'KS',
    r'Kentucky.*': 'KY',
    r'Louisiana.*': 'LA',
    r'Maine.*': 'ME',
    r'Maryland.*': 'MD',
    r'Massachusetts.*': 'MA',
    r'Michigan.*': 'MI',
    r'Minnesota.*': 'MN',
    r'Mississippi.*': 'MS',
    r'Missouri.*': 'MO',
    r'Montana.*': 'MT',
    r'Nebraska.*': 'NE',
    r'Nevada.*': 'NV',
    r'New Hampshire.*': 'NH',
    r'New Jersey.*': 'NJ',
    r'New Mexico.*': 'NM',
    r'New York.*': 'NY',
    r'North Carolina.*': 'NC',
    r'North Dakota.*': 'ND',
    r'Northern Mariana Islands.*':'MP',
    r'Ohio.*': 'OH',
    r'Oklahoma.*': 'OK',
    r'Oregon.*': 'OR',
    r'Pennsylvania.*': 'PA',
    r'Puerto Rico.*': 'PR',
    r'Rhode Island.*': 'RI',
    r'South Carolina.*': 'SC',
    r'South Dakota.*': 'SD',
    r'Tennessee.*': 'TN',
    r'Texas.*': 'TX',
    r'Utah.*': 'UT',
    r'Vermont.*': 'VT',
    r'Virgin Islands.*': 'VI',
    r'Virginia.*': 'VA',
    r'Washington.*': 'WA',
    r'West V.*': 'WV', # Written in different ways
    r'Wisconsin.*': 'WI',
    r'Wyoming.*': 'WY'
}

# Convert states to two letter abbreviations
Data116 = Data116.replace(regex = us_state_abbrev)

# Remove @
Data116 = Data116.replace(regex = {r'^@': ''})

# Remove vacant positions
Data116 = Data116[Data116.Name != "Vacant"]

# Look at the data
Data116

,State,Type,Name,Party,Twitter
0,AL,Senator,Richard Shelby,R,SenShelby
1,AL,Senator,Doug Jones,D,DougJones
2,AL,Representative,"Byrne, Bradley",R,RepByrne
3,AL,Representative,"Roby, Martha",R,RepMarthaRoby
4,AL,Representative,"Rogers, Mike",R,RepMikeRogersAL
...,...,...,...,...,...
536,WI,Representative,"Tiffany, Thomas",R,TomTiffanyWI
537,WI,Representative,"Gallagher, Mike",R,MikeforWI
538,WY,Senator,"Enzi, Mike",R,SenatorEnzi
539,WY,Senator,"Barrasso, John",R,SenJohnBarrasso


It is also seen that there are an inconsistency in the ways the names are written. This is changed so all names are written with the first name first:

In [112]:
Data116['Name'] = [name[1][1:]+ " " +name[0] if len(name) == 2 else name[0] for name in [name.replace(u'\xa0', u'').split(',') for name in Data116.Name]]

### 115<sup>th</sup> congress

Now we move onto the 115th congress. This is data stored in a pdf.table, so for this the `camelot` library is used. 

In [113]:
# Get data
file115 = '../Data/Raw/115_congress_twitter.pdf'

# Read table across all pages
tables = camelot.read_pdf(file115, pages = 'all')

# Convert data to pandas data frame
Data115 = pd.DataFrame(np.concatenate([d.df.drop(0).values for d in tables]), columns=tables[0].df.iloc[0]).drop(columns = "District")

# Align chamber name with the 116 data
rename_chamber = {'Rep.': 'Representative', 'Sen.': 'Senator'}
Data115 = Data115.replace(rename_chamber)

# Align name with the 116 data and store it in one column
Data115["Name"] = Data115["First Name"] + " " + Data115["Last Name"]
Data115 = Data115.drop(columns = ["First Name", "Last Name"])

# Align columns name with the 116 data
Data115 = Data115.rename(columns = {'Title': 'Type', "Twitter Handle": "Twitter"})

### Merge data

Now the two datasets are merged. Here we need to take duplicate acounts into account which accounts for reelections.

In [122]:
# Merge data set
Data_Full = Data115.append(Data116, ignore_index = True)

# Get shape
Data_Full.shape

(1072, 5)

In [115]:
import tweepy
from src.tools.twitter_api2 import auth
api = tweepy.API(auth, wait_on_rate_limit=True)

In [126]:
to_remove = []

for index, handle in enumerate(Data_Full.Twitter):
    try:
        u=api.get_user(handle)
    except Exception:
        to_remove.append(index)

In [128]:
Data_Full.drop(index=to_remove)

,State,Party,Type,Twitter,Name
0,AK,R,Representative,repdonyoung,Don Young
1,AK,R,Senator,lisamurkowski,Lisa Murkowski
2,AK,R,Senator,sendansullivan,Dan Sullivan
3,AL,R,Representative,Robert_Aderholt,Robert Aderholt
4,AL,R,Representative,RepMoBrooks,Mo Brooks
...,...,...,...,...,...
1067,WI,R,Representative,TomTiffanyWI,Thomas Tiffany
1068,WI,R,Representative,MikeforWI,Mike Gallagher
1069,WY,R,Senator,SenatorEnzi,Mike Enzi
1070,WY,R,Senator,SenJohnBarrasso,John Barrasso


In [69]:
Data_Full[(Data_Full.duplicated(subset = ["Twitter"])) & ~(Data_Full.duplicated(subset = ["State", "Twitter"]))]

,State,Party,Type,Twitter,Name
547,AS,R,Representative,RepAmata,Amata Radewagen
648,FL,R,Representative,congbillposey,Bill Posey
699,IL,D,Representative,RepMikeQuigley,Mike Quigley
889,NC,D,Representative,RepDavidEPrice,David Price
1021,UT,R,Representative,RepHalRogers,Rob Bishop


In [70]:
Data_Full[(Data_Full.duplicated(subset = ["Twitter"], keep = 'last')) & ~(Data_Full.duplicated(subset = ["State", "Twitter"], keep = 'last'))]

,State,Party,Type,Twitter,Name
176,IL,D,Representative,RepDavidEPrice,Mike Quigley
233,MD,D,Representative,RepMikeQuigley,Jamie Raskin
297,NC,D,Representative,congbillposey,David Price
472,TX,R,Representative,RepAmata,John Ratcliffe
742,KY,R,Representative,RepHalRogers,Harold Rogers


In [74]:
Data_Full = Data_Full.drop(index=[472, 297, 176, 233, 1021])

0       False
1        True
2       False
3        True
4        True
        ...  
1067    False
1068    False
1069    False
1070    False
1071    False
Length: 1067, dtype: bool

In [81]:
Data_Full = Data_Full.drop_duplicates(subset = ["Twitter"], keep = 'last')

In [85]:
Data_Full = Data_Full.drop_duplicates(subset = ["Name"], keep = 'last')

Add the President

In [91]:
Data_Full = Data_Full.append({'State': None, 'Party': 'R', 'Type': 'POTUS', 'Twitter': 'realDonaldTrump', 'Name': 'Donald J. Trump'}, ignore_index=True)

In [92]:
Data_Full

,State,Party,Type,Twitter,Name
0,AS,R,Representative,RepTomPrice,Aumua Radewagen
1,AZ,R,Senator,JeffFlake,Jeff Flake
2,AZ,R,Senator,SenJonKyl,Jon Kyl
3,CA,D,Representative,reppeteaguilar,Peter Aguilar
4,CA,D,Representative,repcardenas,Tony Cardenas
...,...,...,...,...,...
653,WI,R,Representative,MikeforWI,Mike Gallagher
654,WY,R,Senator,SenatorEnzi,Mike Enzi
655,WY,R,Senator,SenJohnBarrasso,John Barrasso
656,WY,R,Representative,Liz_Cheney,Liz Cheney


In [97]:
Data_Full.to_csv('../Data/Processed/Twitter_Handles.csv')